In [1]:
from IPython.display import Image, display
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from openai import OpenAI
import os
from dotenv import load_dotenv
import ast
import pandas as pd

In [2]:
load_dotenv()
client = OpenAI()

## Image to Keyword

In [3]:
dataset_path =  "products.csv"
df = pd.read_csv(dataset_path)
df.head()

,product_name,product_image,product_url
0,Fidelio Multimedia Cabinet,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/fidelio-multimed...
1,Get Back 2-Seater Sofa,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/get-back-sofa
2,Grantorino 3-Seater Sofa,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/grantorino-sofa
3,Mamy Blue Armchair with Ottoman,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/mamy-blue-armchair
4,Othello Table,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/othello-table


In [31]:
system_prompt = '''
    You are an agent specialized in tagging images of furniture items, decorative items, or furnishings with relevant keywords that could be used to search for these items on a marketplace.
    
    You will be provided with an image and the title of the item that is depicted in the image, and your goal is to extract keywords for only the item specified. 
    
    Keywords should be concise and in lower case. 
    
    Keywords can describe things like:
    - Item type e.g. 'sofa bed', 'chair', 'desk', 'plant'
    - Item material e.g. 'wood', 'metal', 'fabric'
    - Item style e.g. 'scandinavian', 'vintage', 'industrial'
    - Item color e.g. 'red', 'blue', 'white'
    
    Only deduce material, style or color keywords when it is obvious that they make the item depicted in the image stand out.

    Return keywords in the format of an array of strings, like this:
    ['desk', 'industrial', 'metal']
    
'''

def analyze_image(img_url, title):
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": img_url,
                },
            ],
        },
        {
            "role": "user",
            "content": title
        }
    ],
        max_tokens=300,
        top_p=0.1
    )

    return response.choices[0].message.content

In [19]:
keywords = []
for index, row in df.iterrows():
    try:
        keyword_list = analyze_image(row['product_image'], row['product_name'])
        keywords.append(keyword_list)
    except Exception as e:
        keywords.append(f"Error: {str(e)}")
    print(index)

df['keywords'] = keywords

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254


In [ ]:
df.to_csv('product_keywords.csv', index=False)


## Load Products CSV with Keywords

In [3]:

df = pd.read_csv('product_keywords.csv')

df.head()

,product_name,product_image,product_url,keywords
0,Fidelio Multimedia Cabinet,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/fidelio-multimed...,"['multimedia cabinet', 'wood', 'modern', 'brown']"
1,Get Back 2-Seater Sofa,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/get-back-sofa,"['sofa', '2-seater', 'leather', 'brown', 'mode..."
2,Grantorino 3-Seater Sofa,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/grantorino-sofa,"['sofa', '3-seater', 'fabric', 'wood', 'beige'..."
3,Mamy Blue Armchair with Ottoman,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/mamy-blue-armchair,"['armchair', 'ottoman', 'leather', 'blue', 'br..."
4,Othello Table,https://cdn.shopify.com/s/files/1/0093/8033/15...,https://proof.com.sg/products/othello-table,"['table', 'oval', 'marble top', 'wooden base',..."


In [4]:
def convert_string_to_list(string_list):
    try:
        actual_list = ast.literal_eval(string_list)
        return actual_list
    except ValueError as e:
        print(f"Error converting string to list: {e}")
        return None

In [5]:
unique_keywords = set()

for index, row in df.iterrows():
    try:
        keyword_list = convert_string_to_list(row['keywords'])
        if keyword_list:
            unique_keywords.update(keyword_list)
    except Exception as e:
        print(f"Skipping product '{row['product_name']}' at index {index} due to error: {e}")
        continue

unique_keywords_list = list(unique_keywords)
print(unique_keywords_list)
print(len(unique_keywords_list))


Skipping product 'Go Figure' at index 61 due to error: unterminated string literal (detected at line 1) (<unknown>, line 1)
Skipping product 'Light The Way' at index 96 due to error: invalid syntax (<unknown>, line 1)
Skipping product 'Burano Rug' at index 102 due to error: invalid syntax (<unknown>, line 1)
Skipping product 'Turn+' at index 155 due to error: invalid syntax (<unknown>, line 1)
Skipping product 'Sister Louise' at index 162 due to error: invalid syntax (<unknown>, line 1)
['spindle back', 'faux blossom', 'dark brown', 'woven pattern', 'glass top', 'contemporary', 'sectional sofa', 'leather care', 'wicker basket', 'tropical', 'face design', 'gold accent', 'bronze', 'elliptical', 'lattice', 'lounge chair', 'decorative vase', 'figurine', 'teal', 'textured', 'bowl', 'chair', 'stationery holder', 'red cord', 'desk accessories', 'orchid', 'fountain palm', 'small table', 'sculpture', 'cleaning kit', 'amber', 'rug', '2-seater', 'lattice pattern', 'magazine holder', 'modular', 'b

In [6]:
# Count the occurrences of each keyword in the 'keywords' column of the dataframe
keyword_counts = {}
for keyword in unique_keywords_list:
    count = df['keywords'].str.contains(keyword, regex=False).sum()
    keyword_counts[keyword] = count

# Convert the dictionary to a dataframe for better visualization
df_keyword_counts = pd.DataFrame(list(keyword_counts.items()), columns=['Keyword', 'Count']).sort_values(by='Count', ascending=False)
df_keyword_counts[:15]


,Keyword,Count
104,metal,78
213,modern,70
149,wood,62
224,table,50
210,fabric,48
5,contemporary,46
21,chair,40
138,white,39
168,black,37
242,brown,36


## Embedding Keywords

In [9]:
def get_embedding(value, model="text-embedding-3-large"): 
    embeddings = client.embeddings.create(
      model=model,
      input=value,
      encoding_format="float"
    )
    return embeddings.data[0].embedding

In [13]:
df_keywords = pd.DataFrame(unique_keywords_list, columns=['keyword'])
df_keywords['embedding'] = df_keywords['keyword'].apply(lambda x: get_embedding(x))
df_keywords

,keyword,embedding
0,lantern,"[-0.0069259447, -0.03568734, -0.0018425976, 0...."
1,dining table,"[-0.013363886, 0.0036296975, -0.009659945, 0.0..."
2,freestanding,"[-0.015173343, 0.008831116, -0.010515562, -0.0..."
3,mirror,"[-0.03284055, -0.0026042038, -0.008534649, 0.0..."
4,with shelf,"[-0.055056524, 0.009057853, -0.025313543, -0.0..."
...,...,...
260,dark brown,"[-0.0032348004, 0.009423737, -0.0022780863, -0..."
261,area rug,"[-0.037138514, -0.018781073, -0.000190304, 0.0..."
262,crystal,"[-0.0039020386, -0.003952973, -0.0056913956, 0..."
263,wine bag,"[-0.03963623, 0.0102983415, -0.003990165, 0.00..."


### for embeddings of new products/keywords

In [14]:
def compare_keyword(keyword):
    embedded_value = get_embedding(keyword)
    df_keywords['similarity'] = df_keywords['embedding'].apply(lambda x: cosine_similarity(np.array(x).reshape(1,-1), np.array(embedded_value).reshape(1, -1)))
    most_similar = df_keywords.sort_values('similarity', ascending=False).iloc[0]
    return most_similar

def replace_keyword(keyword, threshold = 0.6):
    most_similar = compare_keyword(keyword)
    if most_similar['similarity'] > threshold:
        print(f"Replacing '{keyword}' with existing keyword: '{most_similar['keyword']}'")
        return most_similar['keyword']
    return keyword

## Demo

In [7]:
def search_products_by_keyword(df, keyword):
    # Filter the dataframe for products containing the keyword in their keywords list
    filtered_df = df[df['keywords'].apply(lambda x: keyword in x)]
    # Return the first 5 products
    return filtered_df

def render_products(df):
    # Render the first 5 products, images, and keywords
    for index, row in df.iterrows():
        product_name = row['product_name']
        product_url = row['product_url']
        product_image = row['product_image']
        product_keywords = row['keywords']
        print(f"Product Name: {product_name}")
        print(f"Product URL: {product_url}")
        print(f"Keywords: {product_keywords}")
        display(Image(url=product_image))
        print("\n")


In [8]:
keyword = "metal"
number = 10

filtered_df = search_products_by_keyword(df, keyword)
print(f"There are {len(filtered_df)} products that contain the keyword '{keyword}' \nHere are {number} of them:\n\n")

render_products(filtered_df[:number])

There are 78 products that contain the keyword 'metal' 
Here are 10 of them:


Product Name: Bob Coffee Table
Product URL: https://proof.com.sg/products/bob-coffee-table
Keywords: ['coffee table', 'round', 'metal', 'red']




Product Name: Fidelio Low Cabinet
Product URL: https://proof.com.sg/products/fidelio-low-storage-cabinets
Keywords: ['low cabinet', 'contemporary', 'metal', 'beige']




Product Name: Mr Moonlight Bed
Product URL: https://proof.com.sg/products/pf3211d5615272
Keywords: ['bed', 'upholstered', 'contemporary', 'metal legs']




Product Name: Jay Lounge Reclining Armchair with Ottoman
Product URL: https://proof.com.sg/products/reclining-armchair-ottoman
Keywords: ['reclining armchair', 'ottoman', 'leather', 'modern', 'metal', 'grey']




Product Name: XI Table Lamp
Product URL: https://proof.com.sg/products/xi-table-lamp
Keywords: ['table lamp', 'metal', 'glass', 'bronze', 'white']




Product Name: Soffi Composition Chandelier
Product URL: https://proof.com.sg/products/suspension-small-lamp
Keywords: ['chandelier', 'modern', 'glass', 'metal', 'wood', 'gold']




Product Name: Bob Side Table
Product URL: https://proof.com.sg/products/bob-side-table-43-cm
Keywords: ['side table', 'marble top', 'metal base', 'white']




Product Name: Bolero Dining Table
Product URL: https://proof.com.sg/products/bolero-dining-table-poltrona
Keywords: ['dining table', 'wood', 'metal', 'modern', 'rectangular', 'dark color']




Product Name: Fidelio Coffee Table
Product URL: https://proof.com.sg/products/fidelio-coffee-table-poltrona
Keywords: ['coffee table', 'marble', 'contemporary', 'metal', 'white']




Product Name: Fred Home Desk
Product URL: https://proof.com.sg/products/fred-home-desk
Keywords: ['desk', 'contemporary', 'metal', 'wood', 'gray', 'black']
